In [1]:
from helpers import plotframes, reconstruct_from_patches, plotframes_tensorboard
from mvt import MaskedVideoTransformer
from gameds import GameDS
import torch
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

/home/constantin/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/constantin/miniconda3/lib/python3.11/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [2]:
device = "cuda"

In [3]:
from torch.utils.data import DataLoader
train_ds = GameDS("sequences", train=True)
train_dataloader = DataLoader(train_ds, batch_size=24, shuffle=True, prefetch_factor=4, num_workers=4)
val_ds = GameDS("sequences", train=False)
val_dataloader = DataLoader(val_ds, batch_size=24, shuffle=True, prefetch_factor=2, num_workers=2)

In [4]:
model = MaskedVideoTransformer()
model.to(device);

In [5]:
current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
experiment_name = f'runs/experiment_10_{current_time}'
writer = SummaryWriter(experiment_name)

In [6]:
import matplotlib.pyplot as plt

In [ ]:
criterion = torch.nn.MSELoss()
optim = torch.optim.Adam(params=model.parameters())

for step, (X, _) in enumerate(train_dataloader):
    X = X / 255
    X = X.to(device)
    
    
    X_pred, (X_masked, ) = model(X)
    
    loss = criterion(X_pred, X)

    loss.backward()
    if step%20 == 1:
        optim.step()
        optim.zero_grad()
    
    writer.add_scalar('Training loss', loss.item(), step)

    if step%100 == 1:
        image_tensor = plotframes_tensorboard(X_masked, "Train: Mask")
        writer.add_image('Train: Mask', image_tensor, step)
        
        image_tensor = plotframes_tensorboard(X_pred, "Train: Prediction")
        writer.add_image('Train: Prediction', image_tensor, step)
    
        image_tensor = plotframes_tensorboard(X, "Train: Original")
        writer.add_image('Train: Original', image_tensor, step)
    
    del X_pred
    del X_masked
    